# BEE 4750 Sea Level Rise Linear Program

**Name**: floating sigmas



## Setup

below are the necessary packages. this code will
1. take in a value for flooding
2. output an optimal solution for the municipality.


In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/4750coding/SLR-LP-4750`


In [2]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver

## Overview
The goal is to 

## Exercise (3 points)


| Flood protection Type | Price/unit | Protection (buildings/unitOfImplementation) |
|:-------------:|:----------------------:|:--------------------:|
|       A      |   50,000    |         10          |
|       B      |   100,000    |         25          |
|       C       |   200,000    |         50          |



objective - minimize buildings lost

variables - how much of each flood proofing to install (A B and C) ((will find real examples later))

constraints - cannot go over the budget



In [6]:
using JuMP
using HiGHS  # HiGHS solver for linear programming

# Parameters
budget = 1_000_000  # Total budget in dollars
flood_level = 5.0   # Example flooding level (e.g., meters)

# Flood damage function coefficients
a = 100  # Base number of flooded buildings per unit flood level
b = 1.2  # Exponent for flood level effect on damage

# Infrastructure options (cost per unit, buildings protected per unit)
infrastructure = [
    (50_000, 10),  # Protection A: $50,000/unit, protects 10 buildings/unit
    (100_000, 25), # Protection B: $100,000/unit, protects 25 buildings/unit
    (200_000, 50)  # Protection C: $200,000/unit, protects 50 buildings/unit
]

# Calculate the total number of buildings at risk based on flood level
D = a * flood_level^b

# Create a JuMP model with HiGHS solver
model = Model(HiGHS.Optimizer)

# Decision variables: amount of protection for each option
@variable(model, x[1:length(infrastructure)] >= 0)

# Objective: Minimize the number of flooded buildings
@objective(model, Min, D - sum(infrastructure[i][2] * x[i] for i in 1:length(infrastructure)))

# Constraint 1: Budget constraint
@constraint(model, sum(infrastructure[i][1] * x[i] for i in 1:length(infrastructure)) <= budget)

# Solve the model
optimize!(model)

# Check and display results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found!")
    println("Amounts of protection to implement:")
    for i in 1:length(infrastructure)
        println("  Protection $(Char('A' + i - 1)): $(value(x[i])) units")
    end
    println("Total flooded buildings: ", D - sum(infrastructure[i][2] * value(x[i]) for i in 1:length(infrastructure)))
else
    println("No optimal solution found.")
end

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e+04, 2e+05]
  Cost   [1e+01, 5e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+06, 1e+06]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-1); columns 0(-3); elements 0(-3) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  4.3986483073e+02
HiGHS run time      :          0.00
Optimal solution found!
Amounts of protection to implement:
  Protection A: 0.0 units
  Protection B: 10.0 units
  Protection C: 0.0 units
Total flooded buildings: 439.8648307306073


NEXT
- find actual types of protection, how much they actually cost and how many 'building equivalents' they actually protect
- find municipality data for NYC for realistic values of budget etc.
- correctly convert the MC Flood value into the buildings lost value 'D' here

## References

Put any consulted sources here, including classmates you worked with/who
helped you.

I worked with Cella Schnabel